# Tutorial : How to handle non-numeric data before regression

In this tutorial we are going to explore the diffirent ways in which we should handle the non-numeric data before feeding into the Regression models. 
The contents of this tutoial aims to provide enough understanding for handling the non-numermic data in your next Kaggle assignment!

Non-numeric(Categorical) data can hold a lot of important information which needs to be used while modelling. However, the problem with cateorical data is that, most of the regression models cannot work directly with them(some kind of transfomation becomes mandatory). 

There are a few functions available in python which can be applied directly to the entire data set, to convert the categorial data into distinct numeric values, but this can lead to a possibility of changing the meaning of the data. So we should avoid it unless we want quick but less accurate results.

We are making use of the Ames Housing dataset as our example. The dataset can be found on the Kaggle webpage - https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data



In [22]:
# reading the data

import pandas as pd
from azureml import Workspace

ws = Workspace()
ds1 = ws.datasets['train.csv']
frame1 = ds1.to_dataframe()

df_train= pd.DataFrame(frame1)

In [23]:
#listing out all the non-numeric columns in the data set
for i in df_train.columns.values:
    if df_train[i].dtype != 'int64' and df_train[i].dtype != 'float64':
        print(i, df_train[i].dtype)

MSZoning object
Street object
Alley object
LotShape object
LandContour object
Utilities object
LotConfig object
LandSlope object
Neighborhood object
Condition1 object
Condition2 object
BldgType object
HouseStyle object
RoofStyle object
RoofMatl object
Exterior1st object
Exterior2nd object
MasVnrType object
ExterQual object
ExterCond object
Foundation object
BsmtQual object
BsmtCond object
BsmtExposure object
BsmtFinType1 object
BsmtFinType2 object
Heating object
HeatingQC object
CentralAir object
Electrical object
KitchenQual object
Functional object
FireplaceQu object
GarageType object
GarageFinish object
GarageQual object
GarageCond object
PavedDrive object
PoolQC object
Fence object
MiscFeature object
SaleType object
SaleCondition object


## The Label Encoder - sklearn.preprocessing.LabelEncoder

It is used to transform non-numerical data to numerical data. Numerical labels are always between 0 and (num_of_classes)-1. 

Lets take the example of 'SaleType' column

SaleType: Type of sale
		
       WD 	Warranty Deed - Conventional
       CWD	Warranty Deed - Cash
       VWD	Warranty Deed - VA Loan
       New	Home just constructed and sold
       COD	Court Officer Deed/Estate
       Con	Contract 15% Down payment regular terms
       ConLw	Contract Low Down payment and low interest
       ConLI	Contract Low Interest
       ConLD	Contract Low Down
       Oth	Other
       
This column has 10 diffirent values. Hence it would be labeled from 0 to 9.

In [34]:
#before LabelEncoder
df_le = df_train.loc[:,["Id", 'SaleType']]
df_le.columns = ['Id', 'SaleType']

df_le.head()

,Id,SaleType
0,1,WD
1,2,WD
2,3,WD
3,4,WD
4,5,WD


In [36]:
# Using LabelEncoder to convert categorical data to numeric.
from sklearn.preprocessing import LabelEncoder

le= LabelEncoder()
df_le['SaleType'] = le.fit_transform(df_le['SaleType'].astype('str'))

In [39]:
# After LabelEncoder
df_le.head()

,Id,SaleType
0,1,8
1,2,8
2,3,8
3,4,8
4,5,8


In [46]:
# we can also apply the label encoding transformation to the entire dataset at once.
# No need to specify the column names explicitily.
df_le.apply(LabelEncoder().fit_transform)

df_le.head()


,Id,SaleType
0,1,8
1,2,8
2,3,8
3,4,8
4,5,8


In [50]:
# we can get back the original values by doing a inverse transform as below:
le.inverse_transform(df_le['SaleType'])

array(['WD', 'WD', 'WD', ..., 'WD', 'WD', 'WD'], dtype=object)

### Drawback of LabelEncoder:
A common challenge with nominal categorical variable is that, it may decrease performance of a model. 

For example: 
If we consider feature 'OverallCond'

OverallCond: Rates the overall condition of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average	
       5	Average
       4	Below Average	
       3	Fair
       2	Poor
       1	Very Poor

Even this column has 10 levels/categories. Hence After using the label encoder, our model will get confused and treat OverallCond and SaleType similarly since both will have similar data points. Which is not the corrrect way.

#### Labeling Manually wherever required:
Sometimes its better to manually label the categorical vales into integers so as to maintain the order. For eg. consider the column BsmtCond.

BsmtCond: Evaluates the general condition of the basement

       Ex	Excellent
       Gd	Good
       TA	Typical - slight dampness allowed
       Fa	Fair - dampness or some cracking or settling
       Po	Poor - Severe cracking, settling, or wetness
       NA	No Basement
       
Here, the value Ex should be treated higher than Po or NA. Hence if we are going to label these value, we should make sure that EX receives a higher value. To make sure of this, we label such coolumns manually as follows:


In [12]:
#before Transform
df_train.BsmtQual.head()

0    Gd
1    Gd
2    Gd
3    TA
4    Gd
Name: BsmtQual, dtype: object

In [13]:
df_train = df_train.replace({'BsmtQual': 
                                        {   'Ex': 5, 
                                            'Gd': 4, 
                                            'TA': 3, 
                                            'Fa': 2,
                                            'Po': 1,
                                            'NoBsmt': 0}
                            })

In [14]:
#After Transform
df_train.BsmtQual.head()

0    4.0
1    4.0
2    4.0
3    3.0
4    4.0
Name: BsmtQual, dtype: float64

This way you can preserve the order of the data and the model can understand the order as well.

# Using Dummy variables - Pandas.get_dummies()



This method expands the dimensions by creating new columns for each category of the variable. This method ensures that no weights get assigned to the categories while tranforming them. It transforms each categorical varible with n categories into n binary variables, with only one active. This method is also known as One-Hot Encoding

Lets take the simple example of Street

Street: Type of road access to property

       Grvl	Gravel	
       Pave	Paved

This variable has 2 categories. Gravel and Paved. Hence this function will replace the exisiting variable wih 2 variables- Street_Gravel and Street_Pave. These new variables will have binary data indicating the category type.


In [54]:
df_dummies = df_train.loc[:,["Id", 'Street']]
df_dummies.columns = ['Id', 'Street']

df_dummies.head()

,Id,Street
0,1,Pave
1,2,Pave
2,3,Pave
3,4,Pave
4,5,Pave


In [56]:
#this creates dummy columns for each categorical variable leaving the numerical columns as it is.
df_dummies = pd.get_dummies(df_dummies)

df_dummies.head()

,Id,Street_Grvl,Street_Pave
0,1,0,1
1,2,0,1
2,3,0,1
3,4,0,1
4,5,0,1


In [66]:
# get_dummies can be applied over the entire dataset at once.
# it will leave the numeric variables as it is and create dummy variables for non-nuremic variables.
df_temp = df_train.copy()

# Before creating dummies
df_temp.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [67]:
# After creating dummies 
df_temp = pd.get_dummies(df_temp)

df_temp.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,1,0,0,0,0,1,0


##### Advantages of using One hot encoding(dummies) over LabelEncoding:

Algorithms like linear models (such as logistic regression) learn a single weight for each feature.
This can cause a problem when using LabelEncoder as is assigns weights to each category. This problem won't occur for One hot encoding as it expands the dimentionality by creating new variables and hence each value will be treated equally.

##### Disdvantage of using One hot encoding(dummies):
Since it expands the dimentionality, it requires a lot of memory. It could cause a problem for large datasets. This problem won't occur in LabelEncoding as it just replaces the exisiting values with integers.





# Conclusion:
In this tutorial, we have covered the 2 widely used techniques for transforming non-numeric data:
    1. Using LabelEncoder
    2. Creating dummy variables

Also we have discussed the advantages and disadvantages of both.    
    
This tutorial should provide a breif understanding of the 2 techiques 

